- Dict with top models for each horizon
- Trainflow where number of runs equals horizon
- Append each training day to master list for performance tracking?
- Pull in performance over last n_days (120 min?, 30 and 10?)
    - Focused on precision
- Also ensemble scoring

In [32]:
import performance_flow
import importlib
importlib.reload(performance_flow)
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
#df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
#past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
#past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
#past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone

# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Index(['Date', 'Close', 'High', 'Low', 'Volume', 'Past_Return%_1',
       'Past_Return%_2', 'Past_Return%_3', 'Past_Return%_5', 'Past_Return%_10',
       'Past_Return%_20', 'Past_Return%_30', 'Sum10_Past_Return_1',
       'Sum10_Past_Return_2', 'Sum10_Past_Return_3', 'Sum10_Past_Return_5',
       'Sum10_Past_Return_10', 'Sum10_Past_Return_20', 'Sum10_Past_Return_30'],
      dtype='object')
Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


In [ ]:
feature_dict['past_return']

In [23]:
# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "horizon_30_baseline_new.csv" # Match prior cell saved as file name horizon_2_baseline_new
min_th = 0.55
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_main)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_main, perf_df, returns, min_th)
composite_score.head(6)#.sort_values(by='nprec', ascending=False).head(5)

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
82,30,xgboost-3,5,ma_sma-experimental_slope,1.5-r,4,0.81,117.0,0.72,0.73,0.20,3.29,0.54,0.99,0.45,0.95,0.98,0.47,0.67
69,30,xgboost-3,5,ma_lag-experimental_slope,1.5-r,4,0.83,116.0,0.71,0.69,0.19,4.13,0.53,0.97,0.45,0.94,0.97,0.42,0.65
63,30,xgboost-3,5,ma_lag-ma_sma,1.5-r,4,0.83,116.0,0.70,0.71,0.18,4.63,0.51,0.96,0.45,0.96,0.96,0.46,0.65
81,30,xgboost-3,5,ma_sma-vix_skew,1.5-r,4,0.82,113.0,0.70,0.72,0.19,1.98,0.50,0.97,0.42,0.89,0.97,0.47,0.64
25,30,xgboost-3,6,ma_sma_ba,1.5,100,0.83,117.0,0.69,0.67,0.23,5.00,0.49,0.99,0.39,0.94,0.98,0.36,0.63
45,30,xgboost-3,6,ma_sma_ba,1.5,6,0.84,118.0,0.68,0.71,0.18,2.42,0.45,0.95,0.41,0.90,0.97,0.44,0.62


In [ ]:
"""
2 horizon - man_num, experimental_slope, ma_lag-rsi_acd
5 horizon - past_ret_cols, ma_num
10 horizon - atr_adxvola, ma_lag-vix_skew, ma_lag-ma_sma
20 horizon - ma_lag-ma_num, ma_rel
30 horizon - ma_sma-exp_slope, ma_lag-exp_slope, ma_lag-ma_sma
"""